In [9]:
#Import required modules
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from nltk.cluster import KMeansClusterer, \
    cosine_distance,\
    euclidean_distance
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from nltk.corpus import stopwords
from sklearn import metrics
import numpy as np
from sklearn import mixture
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import pairwise_distances

In [10]:
#Read the train and test sets 
#Train set is a subset of the dataset (5090 Reviews)
train = pd.read_csv("C:/Sem 2/web mining/git clone/BIA-660-main/dataset/train1.csv")
train.head()
train.shape
#Test set is manually labeled subset of data(100 reviews)
test = pd.read_csv("C:/Sem 2/web mining/project/test_set.csv")
test.head()

,Date,Resturant Name,Rating,Review,Source
0,11/7/2021,10Th And Willow Bar And Grill Hoboken,5,loved place much appetizer look great decided ...,Yelp
1,11/1/2021,10Th And Willow Bar And Grill Hoboken,1,terrible wing recommended go get wing got wate...,Yelp
2,7/7/2021,10Th And Willow Bar And Grill Hoboken,4,standard looking american bar inside tv sport ...,Yelp
3,6/14/2021,10Th And Willow Bar And Grill Hoboken,5,place n't get 5 star rating every patron walk ...,Yelp
4,5/2/2021,10Th And Willow Bar And Grill Hoboken,3,nothing special would really give 2 5 star cou...,Yelp


(5093, 5)

,Unnamed: 0,Resturant Name,Review,label
0,2,10Th And Willow Bar And Grill Hoboken,standard looking american bar inside tv sport ...,0
1,355,Acai Ya Later Hoboken,lunch two gentleman helped super sweet great c...,0
2,559,Ainsworth Hoboken Hoboken,sport bar sure food average tv situation sport...,0
3,1161,Ali Baba Hoboken 2,great service warm cozy feeling place ordered ...,7
4,1434,Amandas Restaurant Hoboken 2,stunning one best reataurants ever extremely g...,7


In [11]:
test_text = test["Review"]
test_label = test["label"]
print(test_text)
print(test_label)

0      standard looking american bar inside tv sport ...
1      lunch two gentleman helped super sweet great c...
2      sport bar sure food average tv situation sport...
3      great service warm cozy feeling place ordered ...
4      stunning one best reataurants ever extremely g...
                             ...                        
98     lot choice snack premise juice bowl meal prepa...
99     awesome high quality food chicken organic salm...
100    shaka bowl offer nice space inside outside esp...
101    love shaka bowli moved hoboken upper east side...
102    ordered acai bowl disappointed base tasteless ...
Name: Review, Length: 103, dtype: object
0       0
1       0
2       0
3       7
4       7
       ..
98     14
99     14
100    14
101    14
102    14
Name: label, Length: 103, dtype: int64


In [12]:
train_text= train["Review"] 
print(train_text)

0       loved place much appetizer look great decided ...
1       terrible wing recommended go get wing got wate...
2       standard looking american bar inside tv sport ...
3       place n't get 5 star rating every patron walk ...
4       nothing special would really give 2 5 star cou...
                              ...                        
5088    benny tudino definitely hoboken staple everyon...
5089    fun place small family oriented pizza amazing ...
5090    greate pizzathey accept cash place serf great ...
5091    good thing place delivery quick staff rude del...
5092    pizzeria grandiose sized pizza slice cute ambi...
Name: Review, Length: 5093, dtype: object


In [13]:
#Function for KMeans and GMM clustering algorithms
def clustering(train_text, test_text, test_label, num_clusters, model='KMeans', metric='Cosine'):
    
    clusterer, tfidf_vect = None, None
    
    tfidf_vect = TfidfVectorizer(stop_words="english",min_df=4) 
    dtm= tfidf_vect.fit_transform(train_text)
    test_dtm = tfidf_vect.transform(test_text)
        #print (dtm.shape)
    if model=='KMeans':
        #K-means
        
        if metric=='Cosine':
            clusterer = KMeansClusterer(num_clusters,cosine_distance,repeats=22)
        elif metric=='L2':
            clusterer = KMeansClusterer(num_clusters,euclidean_distance,repeats=22)
            
        clusters = clusterer.cluster(dtm.toarray(), assign_clusters=True)
        #print(clusters[0:5])

        test_dtm = tfidf_vect.transform(test_text)
        predicted = [clusterer.classify(v) for v in test_dtm.toarray()]
        #print(predicted[0:10])


        confusion_df = pd.DataFrame(list(zip(test_label.values, predicted)),columns = ["label", "cluster"])
        #print(confusion_df.head())

        # generate crosstab between clusters and true labels
        confusion = pd.crosstab( index=confusion_df.cluster, columns=confusion_df.label)

        mapping_id = confusion.idxmax(axis=1)

        predicted_target=[mapping_id[i] for i in predicted]
        print(metrics.classification_report(test_label, predicted_target))
        

    elif model=='GMM':
        lowest_bic = np.infty   # initial BIC is set to infinity
        best_gmm = None

        n_components_range = range(2,6)    # The number of clusters

        cv_types = ['spherical', 'tied', 'diag']
        
        for cv_type in cv_types:
            for n_components in n_components_range:
                # Fit a Gaussian mixture with EM
                gmm = mixture.GaussianMixture(n_components=n_components,
                                              covariance_type=cv_type, random_state=42)
                gmm.fit(dtm.toarray())

                bic = gmm.bic(dtm.toarray())  # get Model BIC

                if bic < lowest_bic:
                    # save the model with lowest BIC sofar
                    lowest_bic = bic
                    best_gmm = gmm

        print(best_gmm)
        print(lowest_bic)

        predicted = best_gmm.predict(test_dtm.toarray())
        #print(predicted[0:5])

        confusion_df = pd.DataFrame(list(zip(test_label.values, predicted)),                            columns = ["label", "cluster"])

        #print(confusion_df.head())

        # generate crosstab between clusters and true labels
        confusion = pd.crosstab( index=confusion_df.cluster, columns=confusion_df.label)

        mapping_id = confusion.idxmax(axis=1)

        predicted_target=[mapping_id[i] for i in predicted]
        print(metrics.classification_report(test_label, predicted_target))

In [14]:
#Used KMeans clustering with num_clusters as 15, with metric as cosine distance
#Arrive at the classification Report through external evaluation
clustering(train_text, test_text, test_label, num_clusters=15, model='KMeans',metric = 'Cosine')

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         6
           2       1.00      0.86      0.92         7
           3       0.58      0.88      0.70         8
           4       0.78      1.00      0.88         7
           5       0.88      1.00      0.93         7
           6       0.62      0.83      0.71         6
           7       0.64      0.78      0.70         9
           8       1.00      0.43      0.60         7
           9       0.38      0.71      0.50         7
          10       0.88      1.00      0.93         7
          11       0.00      0.00      0.00         7
          12       0.41      0.88      0.56         8
          13       0.00      0.00      0.00         7
          14       0.50      0.80      0.62         5

    accuracy                           0.63       103
   macro avg       0.51      0.61      0.54       103
weighted avg       0.53   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
